In [63]:
import pandas as pd
import numpy as np
import re
import io
import warnings
import generador
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
warnings.filterwarnings('ignore')

#UTMB17

In [64]:
utmb17=pd.read_csv('../data/utmb/utmb2017.csv')

In [65]:
utmb17.head()

,index,0,1,2,3,4,5
0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,4 - D'HAENE François\nSalomon,SEH,1.0,19:01:54,00:00:00,Francia
2,2,2 - JORNET BURGADA Kilian\nSalomon,SEH,2.0,19:16:59,00:15:05,España
3,3,14 - TOLLEFSON Tim\nHoka,SEH,3.0,19:53:00,00:51:06,Estados Unidos
4,4,7 - THEVENARD Xavier\nAsics,SEH,4.0,20:03:39,01:01:45,Francia


In [66]:
utmb17.drop(0,axis=0,inplace=True)  #Eliminamos primera columna de nulos

columns=['posicion','nombre','cat','pto_cat', 'tiempo','dif','pais']
utmb17.set_axis(columns,axis='columns',inplace=True)  #Modificamos el nombre de las columnas

In [67]:
utmb17.cat.unique()

array(['SEH', 'V1H', 'V2H', 'V1F', 'SEF', 'ESH', 'V3H', 'V2F'],
      dtype=object)

In [68]:

##Modificamos en un solo bucle la columna de cat separando el dato del texto y normalizando el contenido. Además quitamos el float de la columna pto_cat

utmb17['sex']=utmb17.cat
for i in range(1,len(utmb17)+1):
    utmb17.sex[i]=str(utmb17.cat[i])[-1]        #rellenamos la columna sexo con el dato
    utmb17.cat[i]=str(utmb17.cat[i])[:2]        #dejamos solo el dato de categoria en cat
    utmb17.pto_cat[i]=str(utmb17.pto_cat[i])[0] #limpiamos pto_cat
    
    #Normalizamos los datos de sexo
    if utmb17.sex[i]=='n':
        utmb17.sex[i]='UNKWNOW'
    elif utmb17.sex[i]=='H':
        utmb17.sex[i]='M'
    elif utmb17.sex[i]=='n':
        utmb17.sex[i]='UNKWNOW'
    
    #Normalizamos datos en la columna cat
    if utmb17.cat[i]=='SE' or utmb17.cat[i]=='ES':
       utmb17.cat[i]='SENIOR'
    elif utmb17.cat[i]=='V1':
        utmb17.cat[i]='VET-A'
    elif utmb17.cat[i]=='V2':
        utmb17.cat[i]='VET-B'
    elif utmb17.cat[i]=='V3' : 
        utmb17.cat[i]='VET-C'
        
    
        
utmb17.cat.unique()

array(['SENIOR', 'VET-A', 'VET-B', 'VET-C'], dtype=object)

In [69]:
##Modificamos la columna de nombre donde esta el dato del dorsal, nombre y club y lo separamos en diferentes listas que luego metemos en las columnas correspondientes

equipos=[]
nombres=[]
for i in utmb17.nombre:
    try:
        div=str(i).upper().split('\n')
        equipos.append(div[1].split('/')[0].upper())
    except:
        equipos.append('NONE')
    
    
    #Normalizamos el nombre con la estructura Nombre APELLIDO
    div_nom=div[0].split('-')
    limpiar_nombre=div_nom[1].split()
    temp=[]
    temp.append(limpiar_nombre)
    temp[0]=limpiar_nombre[-1].lower().capitalize()
    temp[1:]=limpiar_nombre[:-1]
    nombres.append(" ".join(temp))
    
utmb17.nombre=nombres
utmb17['club']=equipos



In [70]:
utmb17.head(10)

,posicion,nombre,cat,pto_cat,tiempo,dif,pais,sex,club
1,1,François D'HAENE,SENIOR,1,19:01:54,00:00:00,Francia,M,SALOMON
2,2,Kilian JORNET BURGADA,SENIOR,2,19:16:59,00:15:05,España,M,SALOMON
3,3,Tim TOLLEFSON,SENIOR,3,19:53:00,00:51:06,Estados Unidos,M,HOKA
4,4,Xavier THEVENARD,SENIOR,4,20:03:39,01:01:45,Francia,M,ASICS
5,5,Jim WALMSLEY,SENIOR,5,20:11:38,01:09:44,Estados Unidos,M,HOKA
6,6,Pau CAPELL,SENIOR,6,20:12:43,01:10:49,España,M,THE NORTH FACE
7,7,Dylan BOWMAN,SENIOR,7,20:19:48,01:17:54,Estados Unidos,M,THE NORTH FACE
8,8,Gediminas GRINIUS,SENIOR,8,21:24:19,02:22:25,Lituania,M,VIBRAM
9,9,Zach MILLER,SENIOR,9,21:28:32,02:26:38,Estados Unidos,M,THE NORTH FACE
10,10,Jordi GAMITO,SENIOR,1,21:44:31,02:42:37,España,M,COMPRESSPORT


In [71]:
utmb17=utmb17.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','pto_cat'])  #Recolocamos las columnas

utmb17.reset_index(drop=True,inplace=True)  #Reseteamos el index


In [72]:
utmb17.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687 entries, 0 to 1686
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   posicion  1687 non-null   int64 
 1   tiempo    1687 non-null   object
 2   dif       1687 non-null   object
 3   nombre    1687 non-null   object
 4   club      1687 non-null   object
 5   pais      1687 non-null   object
 6   sex       1687 non-null   object
 7   cat       1687 non-null   object
 8   pto_cat   1687 non-null   object
dtypes: int64(1), object(8)
memory usage: 118.7+ KB


In [73]:
#Modificamos la columna de idiomas que no está totalmente correcta

utmb17.pais[utmb17.pais=='HK']='Hong Kong'
utmb17.pais[utmb17.pais=='TW']='Taiwan'

In [74]:
#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

utmb17.drop(['pto_cat'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes

utmb17['carrera']=generador.carrera(len(utmb17),'utmb')
utmb17['anio']=generador.anio(len(utmb17),'2017')

utmb17=utmb17.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos

In [75]:
utmb17.to_csv('../data/utmb/utmb2017mod.csv', index=False)      #cargamos csv

#UTMB18

In [76]:
utmb18=pd.read_csv('../data/utmb/utmb2018.csv')

In [77]:
utmb18.drop(0,axis=0,inplace=True)  #Eliminamos primera columna de nulos

columns=['posicion','nombre','cat','pto_cat', 'tiempo','dif','pais']
utmb18.set_axis(columns,axis='columns',inplace=True)  #Modificamos el nombre de las columnas

In [78]:
utmb18.cat.unique()

array(['SEH', 'V1H', 'V1F', 'SEF', 'V2F', 'ESH', 'V2H', nan, 'V3H', 'V4F'],
      dtype=object)

In [79]:
##Modificamos en un solo bucle la columna de cat separando el dato del texto y normalizando el contenido. Además quitamos el float de la columna pto_cat

utmb18['sex']=utmb18.cat
for i in range(1,len(utmb18)+1):
    utmb18.sex[i]=str(utmb18.cat[i])[-1]        #rellenamos la columna sexo con el dato
    utmb18.cat[i]=str(utmb18.cat[i])[:2]        #dejamos solo el dato de categoria en cat
    utmb18.pto_cat[i]=str(utmb18.pto_cat[i])[0] #limpiamos pto_cat
    
    #Normalizamos los datos de sexo
    if utmb18.sex[i]=='n':
        utmb18.sex[i]='UNKWNOW'
    elif utmb18.sex[i]=='H':
        utmb18.sex[i]='M'
        
    
    
    
    
    #Normalizamos datos en la columna cat
    if utmb18.cat[i]=='SE' or utmb18.cat[i]=='ES':
       utmb18.cat[i]='SENIOR'
    elif utmb18.cat[i]=='V1':
        utmb18.cat[i]='VET-A'
    elif utmb18.cat[i]=='V2':
        utmb18.cat[i]='VET-B'
    elif utmb18.cat[i]=='V3' or utmb18.cat[i]=='V4': 
        utmb18.cat[i]='VET-C'
    elif utmb18.cat[i]=='na':
        utmb18.cat[i]='UNKWNOW'
utmb18.cat.fillna(value='UNKWNOW')
        
utmb18.cat.unique()

array(['SENIOR', 'VET-A', 'VET-B', 'UNKWNOW', 'VET-C'], dtype=object)

In [80]:
##Modificamos la columna de nombre donde esta el dato del dorsal, nombre y club y lo separamos en diferentes listas que luego metemos en las columnas correspondientes

equipos=[]
nombres=[]
for i in utmb18.nombre:
    try:
        div=str(i).split('\n')
        equipos.append(div[1].split('/')[0].upper())
    except:
        equipos.append('NONE')
    
    
    #Normalizamos el nombre con la estructura Nombre APELLIDO
    div_nom=div[0].upper().split('-')
    limpiar_nombre=div_nom[1].split()
    temp=[]
    temp.append(limpiar_nombre)
    temp[0]=limpiar_nombre[-1].lower().capitalize()
    temp[1:]=limpiar_nombre[:-1]
    nombres.append(" ".join(temp))
    
utmb18.nombre=nombres
utmb18['club']=equipos

In [81]:
utmb18=utmb18.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','pto_cat'])  #Recolocamos las columnas

utmb18.reset_index(drop=True,inplace=True)  #Reseteamos el index


In [82]:
#Modificamos la columna de idiomas que no está totalmente correcta

utmb18.pais[utmb18.pais=='HK']='Hong Kong'
utmb18.pais[utmb18.pais=='TW']='Taiwan'
utmb18.pais[utmb18.pais=='MO']='Moldavia'

In [83]:
#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

utmb18.drop(['pto_cat'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes

utmb18['carrera']=generador.carrera(len(utmb18),'utmb')
utmb18['anio']=generador.anio(len(utmb18),'2018')


utmb18=utmb18.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos

In [84]:
utmb18.to_csv('../data/utmb/utmb2018mod.csv', index=False)      #cargamos csv

#UTMB19

In [85]:
utmb19=pd.read_csv('../data/utmb/utmb2019.csv')

In [86]:
utmb19.head()

,index,0,1,2,3,4,5
0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2 - CAPELL Pau\nTHE NORTH FACE,SEH,1.0,20:19:07,00:00:00,España
2,2,5 - THEVENARD Xavier,SEH,2.0,21:07:56,00:48:49,Francia
3,3,63 - HAWKER Scott\nTEAM VIBRAM,SEH,3.0,21:48:04,01:28:57,Nueva Zelandia
4,4,11 - OWENS Tom\nSALOMON,SEH,4.0,22:04:29,01:45:22,Reino Unido


In [87]:
utmb19.drop(0,axis=0,inplace=True)  #Eliminamos primera columna de nulos

columns=['posicion','nombre','cat','pto_cat', 'tiempo','dif','pais']
utmb19.set_axis(columns,axis='columns',inplace=True)  #Modificamos el nombre de las columnas

In [88]:
##Modificamos en un solo bucle la columna de cat separando el dato del texto y normalizando el contenido. Además quitamos el float de la columna pto_cat

utmb19['sex']=utmb19.cat
for i in range(1,len(utmb19)+1):
    utmb19.sex[i]=str(utmb19.cat[i])[-1]        #rellenamos la columna sexo con el dato
    utmb19.cat[i]=str(utmb19.cat[i])[:2]        #dejamos solo el dato de categoria en cat
    utmb19.pto_cat[i]=str(utmb19.pto_cat[i])[0] #limpiamos pto_cat
    
    
    #Normalizamos los datos de sexo
    if utmb19.sex[i]=='n':
        utmb19.sex[i]='UNKWNOW'
    elif utmb19.sex[i]=='H':
        utmb19.sex[i]='M'
    elif utmb19.sex[i]=='n':
        utmb19.sex[i]='UNKWNOW'
    
    #Normalizamos datos en la columna cat
    if utmb19.cat[i]=='SE' or utmb19.cat[i]=='ES':
       utmb19.cat[i]='SENIOR'
    elif utmb19.cat[i]=='V1':
        utmb19.cat[i]='VET-A'
    elif utmb19.cat[i]=='V2':
        utmb19.cat[i]='VET-B'
    elif utmb19.cat[i]=='V3' or utmb19.cat[i]=='V4': 
        utmb19.cat[i]='VET-C'
    elif utmb19.cat[i]=='na':
        utmb19.cat[i]='unkwnow'
utmb19.cat.fillna(value='unkwnow')
        
utmb19.cat.unique()

array(['SENIOR', 'VET-A', 'VET-B', 'VET-C', 'unkwnow'], dtype=object)

In [89]:
##Modificamos la columna de nombre donde esta el dato del dorsal, nombre y club y lo separamos en diferentes listas que luego metemos en las columnas correspondientes

equipos=[]
nombres=[]
for i in utmb19.nombre:
    try:
        div=str(i).split('\n')
        equipos.append(div[1].split('/')[0].upper())
    except:
        equipos.append('NONE')
    
    
    #Normalizamos el nombre con la estructura Nombre APELLIDO
    div_nom=div[0].upper().split('-')
    limpiar_nombre=div_nom[1].split()
    temp=[]
    temp.append(limpiar_nombre)
    temp[0]=limpiar_nombre[-1].lower().capitalize()
    temp[1:]=limpiar_nombre[:-1]
    nombres.append(" ".join(temp))
    
utmb19.nombre=nombres
utmb19['club']=equipos

In [90]:
utmb19=utmb19.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','pto_cat'])  #Recolocamos las columnas

utmb19.reset_index(drop=True,inplace=True)  #Reseteamos el index

In [91]:
#Modificamos la columna de idiomas que no está totalmente correcta


utmb19.pais[utmb19.pais=='TW']='Taiwan'
utmb19.pais[utmb19.pais=='HK']='Hong Kong'


In [92]:
#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

utmb19.drop(['pto_cat'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes

utmb19['carrera']=generador.carrera(len(utmb19),'utmb')
utmb19['anio']=generador.anio(len(utmb19),'2019')

utmb19=utmb19.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos

In [93]:
utmb19.to_csv('../data/utmb/utmb2019mod.csv', index=False)      #cargamos csv

#UTMB20

In [94]:
utmb20=pd.read_csv('../data/utmb/utmb2020.csv')

In [95]:
utmb20.drop(0,axis=0,inplace=True)  #Eliminamos primera columna de nulos

columns=['posicion','nombre','cat','pto_cat', 'tiempo','dif','pais']
utmb20.set_axis(columns,axis='columns',inplace=True)  #Modificamos el nombre de las columnas

In [96]:
##Modificamos en un solo bucle la columna de cat separando el dato del texto y normalizando el contenido. Además quitamos el float de la columna pto_cat

utmb20['sex']=utmb20.cat
for i in range(1,len(utmb20)+1):
    utmb20.sex[i]=str(utmb20.cat[i])[-1]        #rellenamos la columna sexo con el dato
    utmb20.cat[i]=str(utmb20.cat[i])[:2]        #dejamos solo el dato de categoria en cat
    utmb20.pto_cat[i]=str(utmb20.pto_cat[i])[0] #limpiamos pto_cat
    
    #Normalizamos los datos de sexo
    if utmb20.sex[i]=='n':
        utmb20.sex[i]='UNKWNOW'
    elif utmb20.sex[i]=='H':
        utmb20.sex[i]='M'
    elif utmb20.sex[i]=='n':
        utmb20.sex[i]='UNKWNOW'
    
    #Normalizamos datos en la columna cat
    if utmb20.cat[i]=='SE' or utmb20.cat[i]=='ES':
       utmb20.cat[i]='SENIOR'
    elif utmb20.cat[i]=='V1':
        utmb20.cat[i]='VET-A'
    elif utmb20.cat[i]=='V2':
        utmb20.cat[i]='VET-B'
    elif utmb20.cat[i]=='V3' or utmb20.cat[i]=='V4': 
        utmb20.cat[i]='VET-C'
    elif utmb20.cat[i]=='na':
        utmb20.cat[i]='unkwnow'
utmb20.cat.fillna(value='unkwnow')
        
utmb20.cat.unique()

array(['SENIOR', 'VET-A', 'VET-B', 'VET-C', 'unkwnow'], dtype=object)

In [97]:
##Modificamos la columna de nombre donde esta el dato del dorsal, nombre y club y lo separamos en diferentes listas que luego metemos en las columnas correspondientes

equipos=[]
nombres=[]
for i in utmb20.nombre:
    try:
        div=str(i).split('\n')
        equipos.append(div[1].split('/')[0].upper())
    except:
        equipos.append('NONE')
    
    
    #Normalizamos el nombre con la estructura Nombre APELLIDO
    div_nom=div[0].upper().split('-')
    limpiar_nombre=div_nom[1].split()
    temp=[]
    temp.append(limpiar_nombre)
    temp[0]=limpiar_nombre[-1].lower().capitalize()
    temp[1:]=limpiar_nombre[:-1]
    nombres.append(" ".join(temp))
    
utmb20.nombre=nombres
utmb20['club']=equipos

In [98]:
utmb20=utmb20.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','pto_cat'])  #Recolocamos las columnas

utmb20.reset_index(drop=True,inplace=True)  #Reseteamos el index

In [99]:
#Modificamos la columna de idiomas que no está totalmente correcta

utmb20.pais[utmb20.pais=='HK']='Hong Kong'
utmb20.pais[utmb20.pais=='TW']='Taiwan'

In [100]:
#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

utmb20.drop(['pto_cat'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes

utmb20['carrera']=generador.carrera(len(utmb20),'utmb')
utmb20['anio']=generador.anio(len(utmb20),'2020')

utmb20=utmb20.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos

In [101]:
utmb20.to_csv('../data/utmb/utmb2020mod.csv', index=False)      #cargamos csv

#UTMB21

In [102]:
utmb21=pd.read_csv('../data/utmb/utmb2021.csv')

In [103]:
utmb21.drop(0,axis=0,inplace=True)  #Eliminamos primera columna de nulos

columns=['posicion','nombre','cat','pto_cat', 'tiempo','dif','pais']
utmb21.set_axis(columns,axis='columns',inplace=True)  #Modificamos el nombre de las columnas

In [104]:
##Modificamos en un solo bucle la columna de cat separando el dato del texto y normalizando el contenido. Además quitamos el float de la columna pto_cat

utmb21['sex']=utmb21.cat
for i in range(1,len(utmb21)+1):
    utmb21.sex[i]=str(utmb21.cat[i])[-1]        #rellenamos la columna sexo con el dato
    utmb21.cat[i]=str(utmb21.cat[i])[:2]        #dejamos solo el dato de categoria en cat
    utmb21.pto_cat[i]=str(utmb21.pto_cat[i])[0] #limpiamos pto_cat
    
    #Normalizamos los datos de sexo
    if utmb21.sex[i]=='n':
        utmb21.sex[i]='UNKWNOW'
    elif utmb21.sex[i]=='H':
        utmb21.sex[i]='M'
    elif utmb21.sex[i]=='n':
        utmb21.sex[i]='UNKWNOW'
    
    #Normalizamos datos en la columna cat
    if utmb21.cat[i]=='SE' or utmb21.cat[i]=='ES':
       utmb21.cat[i]='SENIOR'
    elif utmb21.cat[i]=='V1':
        utmb21.cat[i]='VET-A'
    elif utmb21.cat[i]=='V2':
        utmb21.cat[i]='VET-B'
    elif utmb21.cat[i]=='V3' or utmb21.cat[i]=='V4': 
        utmb21.cat[i]='VET-C'
    elif utmb21.cat[i]=='na':
        utmb21.cat[i]='unkwnow'
utmb21.cat.fillna(value='unkwnow')
        
utmb21.cat.unique()

array(['SENIOR', 'VET-A', 'VET-B', 'VET-C'], dtype=object)

In [105]:
##Modificamos la columna de nombre donde esta el dato del dorsal, nombre y club y lo separamos en diferentes listas que luego metemos en las columnas correspondientes

equipos=[]
nombres=[]
for i in utmb21.nombre:
    try:
        div=str(i).split('\n')
        equipos.append(div[1].split('/')[0].upper())
    except:
        equipos.append('NONE')
    
    
    #Normalizamos el nombre con la estructura Nombre APELLIDO
    div_nom=div[0].upper().split('-')
    limpiar_nombre=div_nom[1].split()
    temp=[]
    temp.append(limpiar_nombre)
    temp[0]=limpiar_nombre[-1].lower().capitalize()
    temp[1:]=limpiar_nombre[:-1]
    nombres.append(" ".join(temp))
    
utmb21.nombre=nombres
utmb21['club']=equipos

In [106]:
utmb21=utmb21.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','pto_cat'])  #Recolocamos las columnas

utmb21.reset_index(drop=True,inplace=True)  #Reseteamos el index

In [107]:
utmb21.head()

,posicion,tiempo,dif,nombre,club,pais,sex,cat,pto_cat
0,1,20:46:00,00:00:00,François D'HAENE,CMSAB,Francia,M,SENIOR,1
1,2,20:58:31,00:12:31,Dunand,TEAM COMPRESSPORT,Francia,M,SENIOR,2
2,3,21:12:44,00:26:44,Mathieu BLANCHARD,SALOMON,Francia,M,SENIOR,3
3,4,21:38:45,00:52:45,Ludovic POMMERET,HOKA ONE ONE,Francia,M,VET-A,1
4,5,21:52:47,01:06:47,Germain GRANGIER,ON,Francia,M,SENIOR,4


In [108]:
#Modificamos la columna de idiomas que no está totalmente correcta


utmb21.pais[utmb21.pais=='HK']='Hong Kong'

In [109]:
#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

utmb21.drop(['pto_cat'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes

utmb21['carrera']=generador.carrera(len(utmb21),'utmb')
utmb21['anio']=generador.anio(len(utmb21),'2021')

utmb21=utmb21.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos

In [110]:
utmb21.to_csv('../data/utmb/utmb2021mod.csv', index=False)      #cargamos csv

#UTMB22

In [111]:
utmb22=pd.read_csv('../data/utmb/utmb2022.csv')

In [112]:
utmb22.head()

,index,0,1,2,3,4,5
0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1 - JORNET BURGADA Kilian\nNNORMAL,35-39M,1.0,19:49:30,00:00:00,España
2,2,41 - BLANCHARD Mathieu\nSALOMON,35-39M,2.0,19:54:50,00:05:20,Francia
3,3,11 - EVANS Thomas\nADIDAS TERREX / RED BULL,20-34M,1.0,20:34:36,00:45:06,Reino Unido
4,4,3 - WALMSLEY Jim\nHOKA,20-34M,2.0,21:12:12,01:22:42,Estados Unidos


In [113]:
utmb22.drop(0,axis=0,inplace=True)  #Eliminamos primera columna de nulos

columns=['posicion','nombre','cat','pto_cat', 'tiempo','dif','pais']
utmb22.set_axis(columns,axis='columns',inplace=True)  #Modificamos el nombre de las columnas

In [114]:
utmb22.cat.unique()

array(['35-39M', '20-34M', '40-44M', '20-34F', '45-49M', '55-59M',
       '45-49F', '35-39F', '40-44F', '50-54M', '50-54F', '60-64M',
       '65-69M', '55-59F', '70-74M', '60-64F'], dtype=object)

In [115]:
##Modificamos en un solo bucle la columna de cat separando el dato del texto y normalizando el contenido. Además quitamos el float de la columna pto_cat

utmb22['sex']=utmb22.cat
for i in range(1,len(utmb22)+1):
    utmb22.sex[i]=str(utmb22.cat[i])[-1]        #rellenamos la columna sexo con el dato
    utmb22.cat[i]=str(utmb22.cat[i])[:-1]        #dejamos solo el dato de categoria en cat
    utmb22.pto_cat[i]=str(utmb22.pto_cat[i])[0] #limpiamos pto_cat
    
    #Normalizamos los datos de sexo
    if utmb22.sex[i]=='n':
        utmb22.sex[i]='UNKWNOW'
    elif utmb22.sex[i]=='H':
        utmb22.sex[i]='M'
    elif utmb22.sex[i]=='n':
        utmb22.sex[i]='UNKWNOW'
    
    #Normalizamos datos en la columna cat
    if utmb22.cat[i]=='20-34' or utmb22.cat[i]=='35-39':
       utmb22.cat[i]='SENIOR'
    elif utmb22.cat[i]=='40-44' or utmb22.cat[i]=='45-49':
        utmb22.cat[i]='VET-A'
    elif utmb22.cat[i]=='50-54' or utmb22.cat[i]=='55-59':
        utmb22.cat[i]='VET-B'
    elif utmb22.cat[i]=='60-64' or utmb22.cat[i]=='65-69' or utmb22.cat[i]=='70-74': 
        utmb22.cat[i]='VET-C'
    

        
utmb22.cat.unique()

array(['SENIOR', 'VET-A', 'VET-B', 'VET-C'], dtype=object)

In [116]:
##Modificamos la columna de nombre donde esta el dato del dorsal, nombre y club y lo separamos en diferentes listas que luego metemos en las columnas correspondientes

equipos=[]
nombres=[]
for i in utmb22.nombre:
    try:
        div=str(i).split('\n')
        div_equip=div[1].split('/')
        equipos.append(div[1].split('/')[0].upper())
        
    except:
        equipos.append('NONE')
    
    
    #Normalizamos el nombre con la estructura Nombre APELLIDO
    div_nom=div[0].upper().split('-')
    limpiar_nombre=div_nom[1].split()
    temp=[]
    temp.append(limpiar_nombre)
    temp[0]=limpiar_nombre[-1].lower().capitalize()
    temp[1:]=limpiar_nombre[:-1]
    nombres.append(" ".join(temp))
    
    
utmb22.nombre=nombres
utmb22['club']=equipos

In [117]:
utmb22=utmb22.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','pto_cat'])  #Recolocamos las columnas

utmb22.reset_index(drop=True,inplace=True)  #Reseteamos el index

In [118]:
utmb22.head()

,posicion,tiempo,dif,nombre,club,pais,sex,cat,pto_cat
0,1,19:49:30,00:00:00,Kilian JORNET BURGADA,NNORMAL,España,M,SENIOR,1
1,2,19:54:50,00:05:20,Mathieu BLANCHARD,SALOMON,Francia,M,SENIOR,2
2,3,20:34:36,00:45:06,Thomas EVANS,ADIDAS TERREX,Reino Unido,M,SENIOR,1
3,4,21:12:12,01:22:42,Jim WALMSLEY,HOKA,Estados Unidos,M,SENIOR,2
4,5,21:27:50,01:38:20,Zach MILLER,THE NORTH FACE,Estados Unidos,M,SENIOR,3


In [119]:
#Modificamos la columna de idiomas que no está totalmente correcta

utmb22.pais[utmb22.pais=='TW']='Taiwan'
utmb22.pais[utmb22.pais=='HK']='Hong Kong'
utmb22.pais[utmb22.pais=='XK']='Kosovo'

In [120]:
#Normalizamos formato de campos de las columnas añadiendo la configuración final para todas las tablas

utmb22.drop(['pto_cat'],axis='columns',inplace=True) #damos de baja columnas

#Generamos las columnas restanetes

utmb22['carrera']=generador.carrera(len(utmb22),'utmb')
utmb22['anio']=generador.anio(len(utmb22),'2022')

utmb22=utmb22.reindex(columns=['posicion','tiempo','dif','nombre','club','pais','sex','cat','carrera','anio']) #Reorganizamos

In [121]:
utmb22.to_csv('../data/utmb/utmb2022mod.csv', index=False)      #cargamos csv

In [122]:
utmb=pd.concat([utmb17,utmb18,utmb19,utmb20,utmb21,utmb22])

In [123]:
utmb.to_csv('../data/utmb/utmb.csv',index=False, index_label=False)